**Python for Data Analysts**

**Authors:** *Lucia Dini, Daniela Mayorga, Joo Han, Phenihas Krupa, Raviteja Dogiparthi*

# Introduction to the Data Frame

Understanding passenger satisfaction in the airline industry is crucial for airlines to enhance customer experience, improve loyalty and maintain a competitive advantage. The following database contains information on passenger satisfaction about various services offered during air travel and the information is broken down into class, customer type, flight distance, passenger satisfaction, and various service categories which were rated from 0 to 5. 

The database was retrieved from the "Customer Satisfaction in Airline" database page on Kaggle.com and contains data collected from a sample size of **129,880** customers. 
Link to Data Source: https://www.kaggle.com/datasets/yakhyojon/customer-satisfaction-in-airline/data

## Hypothesis

Based on a first look at the dataset, and our personal experiences we hypothesize the following:
- Seat comfort, customer type, and departure/arrival time convenience has the most positive influence and impact on customers' satisfaction with a flight. 
- We also hypothesize that age has no influence on satisfaction. 
- Lastly, the customers who bought Business class tickets would be more likely to have higher dissatisfaction rates than those who bought Economy class tickets.

The aim of this analysis is then to clean the data, view which factors would increase the overall customers' satisfaction, and thus, conclude if our hypothesis proves to be correct.

## Metadata

The source of this database does not directly specify any information regarding the data provided. Therefore based on other databases of a similar kind, logic, and personal experiences, we assigned the following metadata.

| **Field Name**                  | **Description**                                  |
|:---------------------------------|:-------------------------------------------------|
| **satisfaction**                | Passenger satisfaction (satisfied/dissatisfied) |
| **Customer Type**               | Type of customer (Loyal Customer/Disloyal Customer)|
| **Age**                         | Passenger's age                                   |
| **Type of Travel**              | Purpose of travel (Business travel)                |
| **Class**                       | Class of travel (Business/Eco)                     |
| **Flight Distance**             | Distance of the flight (in miles)                  |
| **Seat comfort**                | Rating for seat comfort (0 to 5)                   |
| **Departure/Arrival time convenient** | Rating for departure/arrival time convenience (0 to 5) |
| **Food and drink**              | Rating for food and drink (0 to 5)                 |
| **Gate location**               | Rating for gate location (0 to 5)                  |
| **Inflight wifi service**       | Rating for inflight wifi service (0 to 5)          |
| **Inflight entertainment**      | Rating for inflight entertainment (0 to 5)         |
| **Online support**              | Rating for online support (0 to 5)                 |
| **Ease of Online booking**      | Rating for ease of online booking (0 to 5)         |
| **On-board service**            | Rating for on-board service (0 to 5)               |
| **Leg room service**            | Rating for leg room service (0 to 5)               |
| **Baggage handling**            | Rating for baggage handling (0 to 5)               |
| **Checkin service**             | Rating for check-in service (0 to 5)               |
| **Cleanliness**                 | Rating for cleanliness (0 to 5)                   |
| **Online boarding**             | Rating for online boarding (0 to 5)               |
| **Departure Delay in Minutes**  | Delay in departure time (in minutes)              |
| **Arrival Delay in Minutes**    | Delay in arrival time (in minutes)                |


Additionally, the following grading scale for values rated from 0 to 5 was used.

| **Rating Value** | **Meaning**   |
|:-----------------|:--------------|
| **0**            | Very Poor      |
| **1**            | Poor           |
| **2**            | Neutral        |
| **3**            | Fair           |
| **4**            | Good           |
| **5**            | Very Good      |


# Importing Packages, Libraries, and Data

In [ ]:
# Importing all libraries and packages

import warnings
warnings.filterwarnings("ignore")  
import matplotlib.pyplot as plt  
import numpy as np  
import pandas as pd 
import plotly.express as px
import seaborn as sns
from scipy import stats
from matplotlib.colors import Normalize
from scipy.stats import ttest_ind

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

np.random.seed(42) 

In [ ]:
# Importing Data

data_path = "./data/" # Relative Path

airline_filename = "Invistico_Airline.csv"

airline_df_data = pd.read_csv(data_path + airline_filename)

airline_df = airline_df_data.copy()

# Data Cleaning

## Changing Data Type

In [ ]:
# Viewing the format of the columns in the dataset

airline_df.info()

In [ ]:
# First changing all columns that are categories into categories

for x in ['satisfaction', 'Customer Type', 'Type of Travel', 'Class']:
    airline_df[x]=airline_df[x].str.lower()
    airline_df[x]=airline_df[x].astype('category')

In [ ]:
# Changin all int64 into float64 for later calculations

for col in airline_df :
    if airline_df[col].dtypes == 'int64' :
        airline_df[col]=airline_df[col].astype('float64')

**The dataset is now set up as the correct dtypes.**

## Checking and Removing NaN Values

In [ ]:
airline_df.isna().sum()

**As shown above, there are 393 rows in "Arrival Delay in Minutes" that have NaN values**

In [ ]:
# example of those that are missing values

missing_data=airline_df[airline_df.isna().sum(axis=1)>0]
missing_data.head()

**Since the NaN values are only 393 rows out of the original 129880 (~0.3%), we decided to just remove the data as the overall data loss in minimal**

In [ ]:
airline_nonull = airline_df.dropna()

In [ ]:
# double checking to make sure dropna function worked
airline_nonull.isna().sum()

## Checking and Removing Duplicates

In [ ]:
# Checking for duplicated values

airline_df[airline_df.duplicated()]

**Thankfully, there are no duplicated values.**

## Checking and Removing Unexpected/Incorrect Data

In [ ]:
# First grab all the columns w/ numerical values

numeric_airline_df = airline_df.select_dtypes(np.number)
numeric_airline_df.head()

In [ ]:
# Using histogram to see if there are any visible errors and/or unexptected outliers
numeric_airline_df.hist(figsize=(15,15));

**Based on the histogram above, the column "Departure Delay in Minutes" and "Arrival Delay in Minutes" seems to have a large outlier based by the fact that majority of the data is accumulated around 0-250 but the x-axis goes out to 1500.**

In [ ]:
# This large outlier is further shown in the boxplot below

outlier_col = ['Departure Delay in Minutes', 'Arrival Delay in Minutes']
melted_df = pd.melt(airline_df, value_vars=outlier_col, var_name='Delay Type', value_name='Delay Time')

# Create a box plot
fig = px.box(melted_df, x='Delay Type', y='Delay Time', title="Departure and Arrival Delays")
# Remove comment on line below to see box plot
#fig.show()

**To remove outliers in the two columns, we use the lambda function to keep the flights shorter than 240 minutes (4 hours) as any delay above 4 hours is considered a canceled flight (source,year). Thus, we remove customer's who may have not flown but still gave a review. This is important as they should not be considered in the analysis as their satisfaction would be heavily biased**

In [ ]:
airline_df[outlier_col] = airline_df[outlier_col].applymap(lambda x: x if x < 240 else np.nan)

In [ ]:
# Replotting the histogram for the Delay columns, we see a few more values 
delay = airline_df[outlier_col]
delay.hist(figsize=(10,5));

**After the removing the outliers from the columns, the data stil suggests a heavy right skew; however, we are able to view more of the distribution as well as removing any biased data.**

In [ ]:
# However, we have created new NaN values, shown below
airline_df.isna().sum()

**The total NaN values then becomes 1213 rows. After the initial dropna function we were left with 129487. Thus, we removed an addtional 1213 rows as the decrease of 0.94% is minimal.**

In [ ]:
airline_dropped_outlier = airline_df.dropna()
airline_df = airline_dropped_outlier
airline_df.isna().sum()

**Now we check to make sure there are no unexpected values**

In [ ]:
# creating a sub-table with columns that are expected to have values between 0 and 5
scale_col = airline_df.iloc[:,6:20]

def check_val(col) :
    return (col < 0) | (col > 5)

filtered_scale = scale_col.apply(check_val, axis = 1)
rows_with_true = airline_df[filtered_scale.any(axis=1)]
rows_with_true

**Based on the empty table above, we can conclude that all the values in the sub-table are between 0 and 5.**

**We now check the categorical columns so that they only contain expected values.**

In [ ]:
# First creating a sub-table that only includes category dtype columns
cat_col = airline_df.select_dtypes(include='category')
cat_col.head()

In [ ]:
# Checking all categorical columns for unique values
for col in cat_col :
    print(airline_df[col].unique())

**As shown above, all the categorical columns only include categories that are to be expected.**

# EDA

## Demographic

### Age 

In [ ]:
bins = [0, 10, 14, 18, 30, 40, 50, 60, 70,float('inf')]
labels = ['Child (0-10)', 'Pre-Teen (10-14)', 
          'Teenager (14-18)', 'Young Adult (18-30)', 
          'Adult (30-40)', 'Middle Aged (40-50)', 
          'Senior (50-60)', 'Elderly (60-70)', 
          'Very Elderly (70+)']

In [ ]:
airline_df['Age Group'] = pd.cut(airline_df['Age'], bins=bins, labels=labels)
grouped = airline_df.groupby(['Age Group', 'satisfaction']).size().unstack(fill_value=0) # Any missing value = 0
grouped

In [ ]:
plt.title('Participants per age group')
age_group = grouped.sum(axis = 1).plot(kind = 'bar')
for p in age_group.patches:
    age_group.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points')
plt.ylabel('Frequency')
plt.show()

**As we can see the age demographic spread is mostly between the age of 18 and 60. Ages that are below 18, and possibly ages 60+, would be individuals who flew with their legal guardian, parent, or caretaker and still completed the survey.**

### Overall Satisfaction Distribution

In [ ]:
# Counting the total number of satisfied and dissatisfied passengers
satisfaction_counts = airline_df['satisfaction'].value_counts()

# Plotting the total count into a pie chart
fig, ax = plt.subplots(figsize=(8, 8))
wedges, texts, autotexts = ax.pie(satisfaction_counts, labels=satisfaction_counts.index, autopct=lambda p: '{:.0f} ({:.1f}%)'.format(p * sum(satisfaction_counts) / 100, p),
                                  startangle=90, colors=['lightgreen', 'lightcoral'], textprops=dict(color="k"))
plt.setp(autotexts, size=10, weight="bold")
plt.title('Customer Satisfaction Distribution')
plt.show()

**Given the cleaned data, we see the overall split between the customers who were satisfied and dissatisfied. As we can see, there were 70,637 satisfied customers and 58,192 dissatisfied.**

### Customer Class vs Type of Travel

In [ ]:
grouped_class = airline_df.groupby(['Type of Travel', 'Class']).size().unstack(fill_value=0) # Any missing value = 0
proportions_class = grouped_class.div(grouped_class.sum(axis=1), axis=0)
grouped_perc = grouped_class * 100
pclass = grouped_perc.div(grouped_class.sum(axis=1), axis=0).plot(kind='bar', stacked=True, figsize=(15, 8))

# Assigning the percentage per container in stacked bra chart
for container in pclass.containers:
    pclass.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=15, color='white')
pclass.set_yticks(range(0, 101, 10))
pclass.set_yticklabels([f'{i}%' for i in range(0, 101, 10)])
pclass.set_xticklabels(pclass.get_xticklabels(), rotation = 0)
pclass.set_title('Customer Type of Travel vs Flight Class')
plt.show

**As we can see we see that the demographic of customers who are traveling for business fly in business class 66.3% of the time. On the otherhand, 82.1% of customers fly economy class when flying for personal travel.**

## Correlation Analysis

### Changing to Numerical

In [ ]:
#coverting all the categorical columns into numerical values
satis_dict = {'satisfied':1, 'dissatisfied':0}
type_dict = {'disloyal customer':0, 'loyal customer':1}
class_dict = {'eco':0, 'eco plus':1, 'business':2}
travel_dict = {'personal travel': 0, 'business travel':1}

#copying the data set into a new set and adding the converted numerical categories 
airline_df_num = airline_df.copy()

airline_df_num['satisfaction'] = airline_df_num['satisfaction'].replace(satis_dict).astype('float64')
airline_df_num['Customer Type'] = airline_df_num['Customer Type'].replace(type_dict).astype('float64')
airline_df_num['Class'] = airline_df_num['Class'].replace(class_dict).astype('float64')
airline_df_num['Type of Travel'] = airline_df_num['Type of Travel'].replace(travel_dict).astype('float64')

airline_df_num.head()

**Based of the above new data-set we can conclude that it consist of only numerical values, which can now be used to find correlations between each column.**

### Correlation Matrix

In [ ]:
corr_survey=airline_df_num.select_dtypes(np.number).corr()
levels_corr=np.zeros(shape=corr_survey.shape)

levels=np.arange(-3,4)
ranges=[(-1,-0.7),(-0.7,-0.4),(-0.4,-0.1),(-0.1,0.1),(0.1,0.3),(0.3,0.7),(0.7,1)]

for i,r in enumerate(ranges):
    lower,upper=r
    levels_corr[(corr_survey>=lower) & (corr_survey<upper)] =levels[i]

levels_corr=pd.DataFrame(data=levels_corr,index=corr_survey.index,columns=corr_survey.columns)

norm = Normalize(vmin=-3, vmax=3)

plt.figure(figsize=(15,15))
sns.heatmap(levels_corr, annot=True, cmap='vlag', norm = norm);

| Correlation Value | Interpretation               |
|-------------------|------------------------------|
| +3      | Strong positive correlation  |
| +2      | Moderate positive correlation|
| +1      | Weak positive correlation    |
| 0         | No linear correlation        |
| -1       | Weak negative correlation    |
| -2       | Moderate negative correlation|
| -3       | Strong negative correlation  |

**Based on this correlation matrix, we find that satisfaction has moderate correlations (=2) with class, inflight entertainment, online support, ease of online booking, on-board service, leg room, and online boarding.**

**Other interesting findings include that seat comfort and food & drink are variables that are highly correlated (=3) and that the delays are highly correlated as well.**

**Finally, we are able to disapprove our first hypothesis that Seat comfort, customer type, and departure/arrival time convenience has the most positive influence and impact on customers' satisfaction as these factors had low to no correlation with satisfaction.**

## T-Test

### Comparing Satisfied and Dissatisfied Customers

In [ ]:
# seperating dataset into satisfied and dissatisfied

airline_df_num_diss = airline_df_num[airline_df_num['satisfaction'] == 0].select_dtypes(np.number)
airline_df_num_satis = airline_df_num[airline_df_num['satisfaction'] == 1].select_dtypes(np.number)

In [ ]:
# satisfied descriptive statistics

airline_df_num_satis.describe()

In [ ]:
# dissatisfied descriptive statistics

airline_df_num_diss.describe()

***Comparing Satisfied and Dissatisfied Customers***

**The two tables shown above depicts the descriptive statistics for each aspect of airline service.
For nearly every aspect, satisfied passengers tend to give higher mean ratings compared to dissatisfied passengers.
Lower mean ratings for dissatisfied passengers indicate potential areas of concern that may require attention to improve overall passenger satisfaction.
In summary, the table/graph provides insights into how different aspects of airline services contribute to passenger satisfaction levels. The information can guide airlines in identifying specific areas for improvement to enhance the overall customer experience.**

In [ ]:
t_statistic, p_value = stats.ttest_ind(airline_df_num_satis, airline_df_num_diss)
alpha = 0.05  # significance level

for val in p_value : 
    if val < alpha:
        print("Reject the null hypothesis")
    else:
        print("Fail to reject the null hypothesis")

**The output reveals that the null hypothesis is rejected for every tested column. This implies that for each variable examined, there are statistically significant differences between the two groups (satisfied and dissatisfied).**

**In summary, the output suggests that there are statistically significant variations between the variables in the dataset when comparing the satisfied and dissatisfied groups. Additionally, the reference to a trend in the correlation matrix and descriptive statistical table implies that variables with moderate correlation also exhibit differences in average scores between the dissatisfied and satisfied groups.**

## Closer Look into Correlated Variables

### Class

In [ ]:
# Comparison btwn class and whether or not customer is satisfied or dissatisfied
grouped_class = airline_df.groupby(['Class', 'satisfaction']).size().unstack(fill_value=0)
grouped_perc = grouped_class * 100
pclass = grouped_perc.div(grouped_class.sum(axis=1), axis=0).plot(kind='bar', stacked=True, figsize=(15, 8))

# Assigning the percentage per container in stacked bra chart
for container in pclass.containers:
    pclass.bar_label(container, fmt='%.1f%%', label_type='center', fontsize=15, color='white')
pclass.set_yticks(range(0, 101, 10))
pclass.set_yticklabels([f'{i}%' for i in range(0, 101, 10)])

plt.show()

**According to this bar graph it disproves our hypothesis as it suggests that business class customers have a higher percentage satisfaction than economy and economy plus.**

### Other Correlated Variables

In [ ]:
# Filtering for satisfied passengers 
airline_df_satisf = airline_df[airline_df['satisfaction'] == 'satisfied']

# Selecting relevant columns
service_category = ['Inflight entertainment', 'Online support', 'Ease of Online booking',
                    'On-board service', 'Leg room service', 'Online boarding']

# Creating a new function that changes the numerical rating into categories
def categorize_ratings(ratings):
    if ratings <= 1:
        return 'Unhappy with Category'
    elif 1 < ratings <= 3:
        return 'Neutral'
    else:
        return 'Happy with Category'

# Creating new categorical columns for satisfaction for each service category
for category in service_category:
    satisfaction_column = f'{category} satisfaction'
    airline_df_satisf[satisfaction_column] = airline_df_satisf[category].apply(categorize_ratings)

# Plotting multiple pie charts, one for each service category
fig, axs = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Count of Ratings for each Category based on satisfied passengers')

# Storing legends for later use
legends = []

for i, category in enumerate(service_category):
    satisfaction_column = f'{category} satisfaction'
    data = airline_df_satisf.groupby(satisfaction_column).size()

    ax = axs[i // 3, i % 3]
    wedges, _, autotexts = ax.pie(data, autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'lightgrey', 'lightcoral'])
    ax.set_title(category)

    # Adding a border around the wedges for better separation
    for wedge in wedges:
        wedge.set_edgecolor('white')
    plt.setp(autotexts, size=8, weight="bold")

    # Storing legends for the first pie chart
    if i == 0:
        legend_labels = data.index
        legend = plt.legend(wedges, legend_labels, title='Satisfaction', loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        legends.append(legend)

# Adjusting layout to include the common legend
plt.tight_layout(rect=[0, 0, 0, 1])

# Displaying the common legend without total count
fig.legend(legends[0].legendHandles, [label.get_text() for label in legends[0].get_texts()], title='Satisfaction', loc='center right')

plt.show()

In [ ]:
# Filtering for dissatisfied passenegrs
airline_df_disat = airline_df[airline_df['satisfaction'] == 'dissatisfied']

# Creating new categorical columns for satisfaction for each service category
for category in service_category:
    satisfaction_column = f'{category} satisfaction'
    airline_df_disat[satisfaction_column] = airline_df_disat[category].apply(categorize_ratings)

# Plotting multiple pie charts, one for each service category
fig, axs = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Count of Ratings for each Category based on disatisfied passengers')

# Storing legends for later use
legends = []

for i, category in enumerate(service_category):
    satisfaction_column = f'{category} satisfaction'
    data = airline_df_disat.groupby(satisfaction_column).size()

    ax = axs[i // 3, i % 3]
    wedges, _, autotexts = ax.pie(data, autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'lightgrey', 'lightcoral'])
    ax.set_title(category)

    # Adding a border around the wedges for better separation
    for wedge in wedges:
        wedge.set_edgecolor('white')
    plt.setp(autotexts, size=8, weight="bold")

    # Storing legends for the first pie chart
    if i == 0:
        legend_labels = data.index
        legend = plt.legend(wedges, legend_labels, title='Satisfaction', loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        legends.append(legend)

# Adjusting layout to include the common legend
plt.tight_layout(rect=[0, 0, 0, 1])

# Displaying the common legend without total count
fig.legend(legends[0].legendHandles, [label.get_text() for label in legends[0].get_texts()], title='Satisfaction', loc='center right')

plt.show()

| Rating Range | Meaning               |
|-------------------|------------------------------|
| 0-1      | Unhappy with Category  |
| 2-3      | Neutral|
| 4-5      | Happy with Category    |

**From the two pie charts above, it can be derived that the majority of passengers who reported to be overall dissatisfied with the flight, was not unhappy with the correlated categories but instead had a neutral response. This may suggest that a neutral response to a category leans more towards an overall dissatisfied rating. On the other hand, majority of the response for those satisfied with the flight were happy with the category.**

**We then suggest that future surveys take note of the trend that a rating of 0-3 may still beget a dissatisfaction with the overall flight.**


# Conclusion

**The consumer survey provides insights to enhance the aviation industry's customer experience, revealing overall satisfaction. Our hypothesis regarding seat comfort, customer type, and departure/arrival time having positive correlations with satisfaction was proven negligible (low to no correlation coefficient). Furthermore, in contrary to our expectations, business-class ticket holders exhibit higher satisfaction, challenging assumptions about dissatisfaction among higher-paying customers. Finally, the analysis supports our hypothesis that age has no influence/impact on satisfaction.**

**Moderate positive correlations are evident between satisfaction and service class, online support, booking ease, and online boarding. Thus, we emphasize improving our online services as it may aid in increasing overall satisfaction as well as being relatively easier to do. Additionally, a similar moderate positive correlation was shown between satisfaction and on-board services, legroom, and inflight entertainment which can also be improved over time as long-term investments.**

**In summary, a two-pronged approach addressing short and long-term factors, grounded in strong customer relationships, positions the airline for consistently satisfying customer experiences. Balancing swift service upgrades with enduring product investments targeting fundamental satisfaction drivers is key.**